# 실습: GPT로 prompting 기법들 체험하기

이번 실습에서는 GPT api를 통해 이론 시간에 배운 prompting 기법들을 다룹니다. 먼저 필요한 library들을 설치합니다.

In [ ]:
pip install --upgrade openai


In [ ]:
import openai
from openai.types.chat import ChatCompletionMessage
import re
import os


# API 키를 명시적으로 설정
openai.api_key = "OPENAI_API_KEY"



In [ ]:
open_ai_client = openai.OpenAI(api_key=openai.api_key)
MODEL = "gpt-4"

In [ ]:
from typing import Dict, Any
from openai.types.chat import ChatCompletionMessage

def extract_and_format_answer(answer_message: str) -> int:
    pattern = r"(\d+)번"
    match = re.search(pattern, answer_message)
    if match:
        return int(match.group(1))
    else:
        return -1

def prediction(paragraph: str, question: str, choices: list):
    # 선택지를 문자열로 변환
    choices_str = "\n".join([f"{i + 1}번: {choice}" for i, choice in enumerate(choices)])

    # GPT-4 모델을 사용하여 예측 결과 생성
    prompt = (
        f"지문을 읽고, 선택지 중에 정답을 골라줘. 답은 번호만 말해주면 돼.\n"
        f"지문: {paragraph}\n"
        f"문제: {question}\n"
        f"선택지:\n{choices_str}\n"
        f"정확하게 답을 골라줘."
    )
    messages = [
        {"role": "system", "content": "너는 수능이라는 시험을 푸는 학생이야. 국어 문제는 객관식이고, 정확한 정답을 골라서 고득점을 얻어야 해."},
        {"role": "user", "content": prompt}
    ]

    completion = open_ai_client.chat.completions.create(model=MODEL, messages=messages)
    # 응답 추출 및 포맷팅
    return extract_and_format_answer(completion.choices[0].message.content)

In [ ]:
import json

with open("2023_11_KICE.json", "r") as f:
    korean_2024_problems = json.load(f)

In [ ]:
total_scores = 0
for korean_2024_problem in korean_2024_problems:
    paragraph = korean_2024_problem["paragraph"]
    for problem in korean_2024_problem["problems"]:
        question = problem["question"]
        choices = problem["choices"]
        answer = problem["answer"]
        score = problem["score"]

        # Pass both problem and paragraph to the prediction function
        pred = prediction(paragraph, question, choices)  # Corrected line
        if answer == pred:
            total_scores += score

print(total_scores)

74
